In [1]:
import os
import numpy as np
import pandas as pd
# from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import get_iou_types, evaluate
from models.load import get_trained_model
from utils.eval import get_ap_ar
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from data.load import get_datasets, get_dataloaders
from data.paths import MIMIC_EYE_PATH
from tqdm import tqdm
from utils.train import  get_coco_eval_params
from utils.coco_eval import get_eval_params_dict



## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore")


%matplotlib inline

In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
from enum import Enum

class TrainedModels(Enum):
    object_detection_and_heatmap_generation = "val_ar_0_5102_ap_0_1966_test_ar_0_5504_ap_0_2362_epoch18_02-14-2023 07-03-16_basline_test"
    object_detection_alone = "val_ar_0_5550_ap_0_1937_test_ar_0_5481_ap_0_1928_epoch19_02-14-2023 16-45-12_basline_test"


In [4]:
all_range_iou_thrs = np.array(
    [
        0.00,
        0.05,
        0.1,
        0.15,
        0.2,
        0.25,
        0.3,
        0.35,
        0.4,
        0.45,
        0.5,
        0.55,
        0.6,
        0.65,
        0.7,
        0.75,
        0.8,
        0.85,
        0.9,
        0.95,
        1.0,
    ]
)
normal_iou_thrs = np.array([0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])
score_thresholds = [0.05]


In [5]:
for select_model in tqdm(
    [
        TrainedModels.object_detection_and_heatmap_generation,
        TrainedModels.object_detection_alone,
    ]
):

    for score_thrs in score_thresholds:

        model, train_info, _, _ = get_trained_model(select_model, device,)
        model = model.to(device)
        model.eval()

        setup = train_info.model_setup
        iou_types = get_iou_types(model, setup)

        dataset_params_dict = {
            "MIMIC_EYE_PATH": MIMIC_EYE_PATH,
            "with_fixations": "heatmap-generation" in setup.tasks,
            "bbox_to_mask": False,
            "labels_cols": setup.label_cols,
        }

        detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
            dataset_params_dict=dataset_params_dict,
        )

        train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
            train_dataset, val_dataset, test_dataset, batch_size=setup.batch_size,
        )

        train_coco = None
        train_coco, val_coco, test_coco, _ = get_coco_eval_params(
            train_dataloader,
            val_dataloader,
            test_dataloader,
            detect_eval_dataset,
            setup.iou_thrs,
            setup.use_iobb,
        )

        all_range_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=all_range_iou_thrs,
        )

        all_cat_ids = [None] + [
            detect_eval_dataset.disease_to_idx(d)
            for d in detect_eval_dataset.labels_cols
        ]

        for cat_id in all_cat_ids:
            cat_ids = (
                [
                    detect_eval_dataset.disease_to_idx(d)
                    for d in detect_eval_dataset.labels_cols
                ]
                if cat_id is None
                else [cat_id]
            )

            if not (cat_ids is None):
                all_range_eval_params_dict["bbox"].catIds = cat_ids
                all_range_eval_params_dict["segm"].catIds = cat_ids

            train_evaluator, _ = evaluate(
                setup=setup,
                model=model,
                data_loader=train_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=train_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            test_evaluator, _ = evaluate(
                setup=setup,
                model=model,
                data_loader=test_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=test_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            val_evaluator, _ = evaluate(
                setup=setup,
                model=model,
                data_loader=val_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=val_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            if cat_id is None:
                disease_str = "all"
            else:
                disease_str = detect_eval_dataset.label_idx_to_disease(cat_id)

            save_iou_results(
                train_evaluator,
                f"train_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                test_evaluator,
                f"test_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                val_evaluator,
                f"val_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )

            train_ap_ar = get_ap_ar(
                train_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

            test_ap_ar = get_ap_ar(
                test_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )
            val_ap_ar = get_ap_ar(val_evaluator, areaRng="all", maxDets=10, iouThr=0.5,)

            df = pd.DataFrame(
                [
                    {
                        "dataset": "train",
                        f"AP@[IoBB = 0.50]": train_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": train_ap_ar["ar"],
                    },
                    {
                        "dataset": "test",
                        f"AP@[IoBB = 0.50]": test_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": test_ap_ar["ar"],
                    },
                    {
                        "dataset": "val",
                        f"AP@[IoBB = 0.50]": val_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": val_ap_ar["ar"],
                    },
                ]
            )

            df.to_csv(
                os.path.join(
                    "eval_results", 
                    f"{select_model.value}_{disease_str}_score_thrs{score_thrs}.csv",
                )
            )
            print_title(disease_str)
            print(df)


  0%|          | 0/2 [00:00<?, ?it/s]

Using pretrained backbone. mobilenet_v3
Using SGD as optimizer with lr=0.01
creating index...
index created!
Evaluation:  [  0/133]  eta: 0:24:39  loss: 0.8728 (0.8728)  object-detection_loss_classifier: 0.0889 (0.0889)  object-detection_loss_box_reg: 0.0606 (0.0606)  object-detection_loss_objectness: 0.0261 (0.0261)  object-detection_loss_rpn_box_reg: 0.0041 (0.0041)  heatmap-generation_heatmap_loss: 0.6931 (0.6931)  model_time: 5.3274 (5.3274)  evaluator_time: 0.0191 (0.0191)  time: 11.1204  data: 4.6775  max mem: 557
Evaluation:  [100/133]  eta: 0:03:05  loss: 0.8518 (0.8569)  object-detection_loss_classifier: 0.0782 (0.0838)  object-detection_loss_box_reg: 0.0630 (0.0607)  object-detection_loss_objectness: 0.0157 (0.0167)  object-detection_loss_rpn_box_reg: 0.0027 (0.0026)  heatmap-generation_heatmap_loss: 0.6931 (0.6931)  model_time: 0.2948 (0.3307)  evaluator_time: 0.0192 (0.0176)  time: 5.5030  data: 4.1955  max mem: 573
Evaluation:  [132/133]  eta: 0:00:05  loss: 0.8479 (0.8613

 50%|█████     | 1/2 [2:03:26<2:03:26, 7406.89s/it]

  dataset  AP@[IoBB = 0.50]  AR@[IoBB = 0.50]
0   train          0.112146          0.322959
1    test          0.184145          0.386139
2     val          0.128124          0.361596
Using pretrained backbone. mobilenet_v3
Using SGD as optimizer with lr=0.01
creating index...
index created!
Evaluation:  [  0/133]  eta: 0:09:02  loss: 0.1632 (0.1632)  object-detection_loss_classifier: 0.0845 (0.0845)  object-detection_loss_box_reg: 0.0553 (0.0553)  object-detection_loss_objectness: 0.0189 (0.0189)  object-detection_loss_rpn_box_reg: 0.0045 (0.0045)  model_time: 0.2766 (0.2766)  evaluator_time: 0.0180 (0.0180)  time: 4.0826  data: 2.6823  max mem: 573
Evaluation:  [100/133]  eta: 0:02:02  loss: 0.1691 (0.1590)  object-detection_loss_classifier: 0.0826 (0.0802)  object-detection_loss_box_reg: 0.0657 (0.0621)  object-detection_loss_objectness: 0.0124 (0.0143)  object-detection_loss_rpn_box_reg: 0.0023 (0.0024)  model_time: 0.2655 (0.2766)  evaluator_time: 0.0244 (0.0238)  time: 3.7089  da

100%|██████████| 2/2 [3:19:18<00:00, 5979.41s/it]  

  dataset  AP@[IoBB = 0.50]  AR@[IoBB = 0.50]
0   train          0.078995          0.229595
1    test          0.125009          0.279703
2     val          0.087862          0.261845
